In [1]:
import os
import shutil
from tqdm import tqdm
import time
import cv2
import numpy as np

# Setting Path

In [2]:
T1W_PATH = r"dataset\T1W_brain_pic"
T2F_PATH = r"dataset\T2F_brain_pic"
T2W_PATH = r"dataset\T2W_brain_pic"
MASK_PATH = r"dataset\Mask_brain_pic"
NUMPY_PATH = r"dataset\numpy"

os.mkdir(T1W_PATH) if not os.path.isdir(T1W_PATH) else print("{T1W_PATH} already existed")
os.mkdir(T2F_PATH) if not os.path.isdir(T2F_PATH) else print("{T2F_PATH} already existed")
os.mkdir(T2W_PATH) if not os.path.isdir(T2W_PATH) else print("{T2W_PATH} already existed")
os.mkdir(MASK_PATH) if not os.path.isdir(MASK_PATH) else print("{MASK_PATH} already existed")
os.mkdir(NUMPY_PATH) if not os.path.isdir(NUMPY_PATH) else print("{NUMPY_PATH} already existed")

DATA_PATH = r"dataset\MRI_T1T2T2F_ROI"

# Options

In [3]:
INCLUDE_EMPTY_MASK = True

# Clustering Dataset

In [4]:
def is_img_empty(img):
    if np.any(img >= 1):
        return False
    return True

In [5]:
try:
    for filename in tqdm(os.listdir(DATA_PATH), desc="copy file", mininterval=0.02):
        
        ## example. "0046_20180613_100115_01_ROI.png" to "ROI"
        extention = os.path.splitext(filename)[0].split('_')[-1]    

        srcDir = os.path.join(DATA_PATH,filename)
        destDir = ""

        if extention == "T1W":
            destDir = os.path.join(T1W_PATH,filename)
        elif extention == "T2F":
            destDir = os.path.join(T2F_PATH,filename)
        elif extention == "T2W":
            destDir = os.path.join(T2W_PATH,filename)
        elif extention == "ROI":
            destDir = os.path.join(MASK_PATH,filename)

        shutil.copy(srcDir,destDir)
except Exception as e:
    print(e)

copy file: 100%|██████████| 9424/9424 [00:17<00:00, 550.38it/s]


In [6]:
if not INCLUDE_EMPTY_MASK:

    delete_list = []

    for img in os.listdir(MASK_PATH):
        if img.endswith('_ROI.png'):
            IMG_PATH = os.path.join(MASK_PATH,img)
            image = cv2.imread(IMG_PATH, cv2.IMREAD_GRAYSCALE)

            if is_img_empty(np.array(image)):
                delete_list.append(img[:24] + "ROI.png")
                delete_list.append(img[:24] + "T1W.png")
                delete_list.append(img[:24] + "T2F.png")
                delete_list.append(img[:24] + "T2W.png")

    for target in tqdm(delete_list, desc='delete file with empty mask'):
        
        target_path = ''

        if target.endswith('_ROI.png'):
            target_path = os.path.join(MASK_PATH,target)
        elif target.endswith('_T1W.png'):
            target_path = os.path.join(T1W_PATH,target)
        elif target.endswith('_T2F.png'):
            target_path = os.path.join(T2F_PATH,target)
        elif target.endswith('_T2W.png'):
            target_path = os.path.join(T2W_PATH,target)

        os.remove(target_path)